In [ ]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 9
### Team Member Names: Germain, Padtmesh, Jin
### Team Strategy Chosen: Safe

In [ ]:
def filterByDenom(ticker):
    # print(ticker)
    # print('------------------------')
    # print(ticker.info['currency'])
    # print('------------------------')
    return ticker.info['currency'] != 'USD'

def filterByVol(hist):
    # print(hist['Volume'])
    return  hist['Volume'].mean() < 200000

def filterByTDays(df_hist):
    df = df_hist.copy()
    for month in range(1, 12):
        idx = pd.to_datetime(df.index).month
        df_month = df.loc[idx == month]
        if (len(df_month) < 20):
            df.drop(df.loc[idx == month].index)
    return df



In [ ]:
def getHist(tickers):
    # Set defaults for timeframe
    start_date = '2022-01-01'
    end_date = '2022-11-01' # adjustment for non-inclusive param end

    # Create dataframe to hold historic data from tickers
    df_sv = pd.DataFrame()

    # Get the closing price of each ticker 
    for tick in tickers['Tickers']:
        # Call API for ticker data
        temp = yf.Ticker(tick)

        # Get historical data
        #   Note: the method history() prints out the delisted message
        temp_hist = temp.history(start=start_date, end=end_date, interval='1d')

        # Apply filters
        if (not temp_hist.empty) and (not (filterByDenom(temp) or filterByVol(temp_hist))):
            # Get Closing Price as a DataFrame
            df_sv[tick] = pd.DataFrame(temp_hist['Close'])

    # Remove indeterminate or faulty values
    df_sv.dropna(axis=1,how='all',inplace = True)
    df_sv.dropna(inplace = True)
    # df_sv = filterByTDays(df_sv, start_date, end_date)
    return df_sv

In [ ]:
def sorHistPct(df_pct):
    return df_pct.reindex(df_pct.std().sort_values().index, axis=1)

In [ ]:
# Random simulation
# alternative could be to iterate completely over the searchspace
# however assuming an upperbound of 1000(even 100) nCr with 25 stocks would be too slow
# instead we could generate random portfolios up to 10^x iterations
# depending on x we can approximate it to a decent degree of accuracy

def getReturns(pct, weights):
    return (np.sum(pct.mean() * weights)) * 253

# May have to pivot to dot function if @ operator isnt supported
def getStd(pct, weights):
    return np.sqrt(np.transpose(weights) @ (pct.cov() * 253) @ weights)

def randSimulation(pct):
    n = 10**4 # possibly 5
    ticker_count = len(pct.columns)
    # print(ticker_count)
    list_return = []
    list_std = []
    list_weight = []
    # Generate n random portfolios
    for p in range(n):
        # Get random values between 0 and 1
        # Adjust random values to be from 0.02 to 0.25
        # Use numpy uniform random array generation as range is low and we want to avoid non-concentrated distributions
        # weights = np.random.random(size = ticker_count)  
        weights = np.random.uniform(size= ticker_count)
        # Establishing lower bound for randm portfolio genertion
        low = 0.02
        temp = weights
        temp = (temp/temp.sum()*(1-low*ticker_count))
        weights = temp+low
        # Divide random values by sum to get weights
        # weights /= np.sum(weights) 
        list_weight.append(weights)
        list_return.append(getReturns(pct,weights))
        list_std.append(getStd(pct,weights))

    # Convert list to np arrays
    arr_return = np.array(list_return)
    arr_std = np.array(list_std)
    print('generated weights')
    print(list_weight)
    print(ticker_count)
    print(round(max(arr_return), 4))
    print(round(min(arr_return), 4))
    print(round(max(arr_std), 4))
    print(round(min(arr_std), 4))
    return arr_return, arr_std, list_weight



def graphSimulation(returns, stds, weights):
    maxs_index = np.argmax(returns/stds)
    maxs_std = stds[maxs_index] 
    maxs_return = returns[maxs_index] 
    ms_weight = weights[maxs_index]
    
    minv_index = np.argmin(stds)
    minv_std = stds[minv_index]
    minv_return = returns[minv_index]
    minv_weight = weights[minv_index]

    # print(minv_weight)

    plt.figure(figsize=(20, 10))

    plt.scatter(stds,returns, c=(returns/stds), cmap='viridis', s=12, alpha=0.4)

    plt.scatter(maxs_std,maxs_return,c='r' , s=120,marker='D', label='Maximum Sharpe Ratio Portfolio')
    plt.scatter(minv_std,minv_return,c='b', s=120, marker='D',label='Minimum Volatility Portfolio')

    plt.colorbar(label='Sharpe Ratio')

    plt.legend()
    plt.title(' Volatility-Returns from Portfolio Optimization')
    plt.xlabel('Annualized Portfolio Volatility')
    plt.ylabel('Annualized Portfolio Returns')
    

In [ ]:
# def createPortFinal():


In [ ]:
# def createStocksFinal():


In [ ]:
# Read in csv file containing tickers
tickers = pd.read_csv('Tickers.csv', names = ['Tickers'])
tickers.head()

In [ ]:
# Get Historic Data for each ticker
hist = getHist(tickers)
hist.head()

# Filter data



In [ ]:
# Construct dataframe of percentage change of each stock
hist_pct = hist.pct_change()

# Sort hist_pct based on std
hist_pct = sorHistPct(hist_pct)

# Create new dataframe inplace from the first 25 tickers
hist_pct = hist_pct.iloc[:,:25]
hist_pct.head()

In [ ]:

gen_return, gen_std, gen_weights = randSimulation(hist_pct)
graphSimulation(gen_return, gen_std, gen_weights)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.